----
# Exercise 3: Wildcard

You know the drill. Pick *ONE* of the following exercises to complete.

----
# Exercise 3.1: Implement ODIN for OOD detection
ODIN is a very simple approach, and you can already start experimenting by implementing a temperature hyperparameter in your base model and doing a grid search on $T$ and $\varepsilon$.


In [ ]:
import torch
import torchvision
from torchvision.datasets import FakeData
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn
from torch import optim
import numpy as np
from sklearn import metrics

In [ ]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

fakeset = FakeData(size=1000, image_size=(3, 32, 32), transform=transform)
fakeloader = torch.utils.data.DataLoader(fakeset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# A very simple CNN model.
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# USE THIS CELL TO LOAD THE PRETRAINED MODEL.
device = 'cpu'
loss = nn.CrossEntropyLoss()
model = CNN().to(device)
model.load_state_dict(torch.load('./OOD_CNN.pth',map_location=torch.device('cpu'))) #50 epochs

In [ ]:
def max_softmax(logit, T=1.0, delta= 0.5):
    s = F.softmax(logit/T, 1)
    s = s.max(dim=1)[0] #get the max for each element of the batch
    return s
    


def compute_scores(data_loader, score_fun):
    scores = []
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            output = model(x.to(device))
            s = score_fun(output)
            scores.append(s)
        scores_t = torch.cat(scores)
        return scores_t


def OOD_det(model, data_loader, T, eps, delta):
    score = []
    for data in data_loader:
        model.train()
        x,y = data
        
        x.requires_grad=True 

        x.retain_grad()

        output=model(x)
        model.zero_grad()
        l = loss(output,y)
        l.backward()
        v = x + eps*torch.sign(x.grad)#fgsm
        with torch.no_grad():
            logit = model(v)
            s = max_softmax(logit, T, delta)
            for i in range(np.shape(s)[0]): #calcolo della funzione g
                if s[i]>delta:
                    score.append(1)
                else:
                    score.append(0)
    score_t = torch.tensor(score)
    return score_t






In [ ]:
##GRID SEARCH ON EPS, T, DELTA(?). Uso test set come validation set.


e = np.linspace(0, 0.004, 10)#10 valori da 0 a 0.004 come nel paper su ODIN
t = np.array([1, 5, 10, 50, 100, 500, 1000])

for eps in e:
    for temp in t:
        scores_fake= OOD_det(model, fakeloader, temp, eps, delta=0.5)
        scores_test= OOD_det(model, testloader, temp, eps, delta=0.5)
        y_test = torch.ones_like(scores_test)
        y_fake = torch.zeros_like(scores_fake)
        y = torch.cat((y_test, y_fake))
        y_pred = torch.cat((scores_test, scores_fake))
        s = metrics.roc_auc_score(y, y_pred)
        print("AUC for eps = %s, temp = %s is %s" % (eps, temp, s))



----
# Exercise 3.2: Implement JARN
In exercise 2.2 you already implemented Jacobian-regularized learning to make your model more robust to adversarial samples. Add a *discriminator* to your model to encourage the adversarial samples used for regularization to be more *salient*.

See [the JARN paper](https://arxiv.org/abs/1912.10185) for more details.


In [ ]:
import torch
import torchvision
from torchvision.datasets import FakeData
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn
from torch import optim
import numpy as np
from sklearn import metrics

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=8)

fakeset = FakeData(size=1000, image_size=(3, 32, 32), transform=transform)
fakeloader = torch.utils.data.DataLoader(fakeset, batch_size=batch_size,
                                         shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


/home/marco/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [ ]:
# A very simple CNN model.
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Adaptor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,3,1)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.tanh(x)
        return x
     


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_net = nn.Sequential(

            nn.Conv2d(3, 16, 5),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5),
            nn.MaxPool2d(2,2)
            )
        self.linear_layers = nn.Sequential(
            nn.Linear(64*10*10, 400),
            nn.ReLU(),
            nn.Linear(400, 10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.conv_net(x)
        x= torch.flatten(x,1)
        x = self.linear_layers(x)
        return x
        

In [ ]:
# USE THIS CELL TO LOAD THE PRETRAINED MODEL.
device = 'cpu'
loss = nn.CrossEntropyLoss()
model = CNN().to(device)
apt = Adaptor().to(device)
disc = Discriminator().to(device)
#model.load_state_dict(torch.load('./OOD_CNN.pth',map_location=torch.device('cpu'))) #50 epochs

In [ ]:
device = 'cpu'
loss = nn.CrossEntropyLoss()
model = CNN().to(device)
apt = Adaptor().to(device)
disc = Discriminator().to(device)

for data in testloader:
    x, y = data
    break

x1,y1 = x,y
a = 0.0001
u = 1
model1 = model
apt1 = apt
disc_new =disc
opt_model1 = torch.optim.Adam(model1.parameters(), lr=a)
opt_apt = torch.optim.Adam(apt.parameters(), lr=a)
opt_disc_new = torch.optim.Adam(disc_new.parameters(), lr=a)
model1.train()
model.train
apt.train()
disc.train()


x.requires_grad = True
output = model(x)
x.retain_grad()
loss = nn.CrossEntropyLoss()
model.zero_grad()
l = loss(output,y)
l.backward()
jacobian = x.grad
x.requires_grad = False
print(np.shape(jacobian))
generated = apt1(jacobian)
#CALCOLO LOSS ADVERSARIAL SFRUTTANDO DEFINIZIONE DI BCE

loss1 = nn.CrossEntropyLoss()
output1 = model1(x1)
l1 = loss1(output1,y1)
real = torch.ones_like(disc(x1))
fake = torch.zeros_like(disc(generated.detach()))
bce = nn.BCELoss()

lreal = bce(real, disc(x1))
lfake = bce(fake, disc(generated))
l_adv = lreal + lfake
#CALCOLO LOSS DI TRAINING, NOMI LOSS ASSEGNATI SECONDO I PARAMETRI DEL PAPER
l_theta = l1 + u*l_adv   #theta parametro di model 

opt_model1.zero_grad()
l_theta.backward()
opt_model1.step()


generated1 = apt(jacobian.detach())
lreal1 = bce(real,disc(x1.detach()))
lfake1 = bce(fake, disc(generated1))
l_adv1 = lreal1 + lfake1
opt_apt.zero_grad()
l_adv1.backward()
opt_apt.step()
 


lreal_disc = bce(real, disc_new(x1.detach()))
lfake_disc = bce(fake, disc_new(generated1.detach()))
l_adv_disc = lreal_disc + lfake_disc
l_phi = -1*l_adv_disc
opt_disc_new.zero_grad()
l_adv_disc.backward()
opt_disc_new.step()




torch.Size([256, 3, 32, 32])


In [ ]:
for name, param in disc_new.named_parameters():
    print(name, param.grad)

print(jacobian)

conv_net.0.weight tensor([[[[nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan]],

         [[nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan]],

         [[nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan]]],


        [[[nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan]],

         [[nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan],
          [nan, nan, nan, nan, nan]],

         [[nan, nan, nan, nan, nan],
        

In [ ]:
device = 'cpu'
loss = nn.CrossEntropyLoss()
model = CNN().to(device)
apt = Adaptor().to(device)
disc = Discriminator().to(device)



for data in testloader:
    x, y = data
    break

x1 = x.clone()
y1 = y.clone()
a = 0.0001
u = 1
model1 = model
apt1 = apt
disc_new =disc
opt_model = torch.optim.Adam(model.parameters(), lr=a)
opt_apt = torch.optim.Adam(apt.parameters(), lr=a)
opt_disc = torch.optim.Adam(disc.parameters(), lr=a)

model1.train()
model.train
apt.train()
disc.train()


x.requires_grad = True
output = model(x)
x.retain_grad()
loss = nn.CrossEntropyLoss()
model.zero_grad()
l = loss(output,y)
l.backward(retain_graph=True)
jacobian = x.grad
x.requires_grad = False
print(np.shape(jacobian))
generated = apt1(jacobian.detach())
#CALCOLO LOSS ADVERSARIAL SFRUTTANDO DEFINIZIONE DI BCE

real = torch.ones_like(disc(x1))
fake = torch.zeros_like(disc(generated))
bce = nn.BCELoss()
#
lreal = bce(real, disc(x1))
lfake = bce(fake, disc(generated))
l_adv = lreal + lfake
print(l_adv)
#CALCOLO LOSS DI TRAINING, NOMI LOSS ASSEGNATI SECONDO I PARAMETRI DEL PAPER
l_theta = l + u*l_adv   #theta parametro di model 

opt_model.zero_grad()
opt_apt.zero_grad()
l_theta.backward(retain_graph=True)
opt_model.step()
opt_apt.step()

#loss1 = nn.CrossEntropyLoss()
#output1 = model(x1)
#l1 = loss1(output1,y1)

#opt_apt.zero_grad()
#l_adv.backward()
for name, param in apt.named_parameters():
    print(name, param.grad)
#opt_apt.step()
#
##Aggiorno il parametro di disc, ma prima calcolo il valore di generated con i pesi di adaptor aggiornati
#
generated_new = apt(jacobian.detach())
lreal = bce(real, disc(x1))
lfake = bce(fake, disc(generated_new.detach()))
l_adv = lreal + lfake
l_phi = -1*l_adv

opt_disc.zero_grad()
l_phi.backward()
opt_disc.step()

for name, param in disc.named_parameters():
    print(name, param.grad)

torch.Size([256, 3, 32, 32])
tensor(99.9980, grad_fn=<AddBackward0>)
conv.weight tensor([[[[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]]]])
conv.bias tensor([nan, nan, nan])


RuntimeError: all elements of target should be between 0 and 1

In [ ]:
#apt = Adaptor().to('cpu')
#apt1=apt
#apt.train
bce = nn.BCELoss()
#print(x)
#for name, param in apt.named_parameters():
#    if param.requires_grad:
#        print(name, param.data)
#print(type(apt(x)[0][0][0][0].item()))
print(apt.conv.weight.grad)


tensor([[[[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]]]])


In [ ]:

def training_Jarn(dataloader, epochs, model, apt, disc, a, b, c, u):
    opt_model = torch.optim.Adam(model.parameters(), lr=a)
    opt_apt = torch.optim.Adam(apt.parameters(), lr=b)
    opt_disc = torch.optim.Adam(disc.parameters(), lr=c)

    model.train()
    apt.train()
    disc.train()
    for epoch in range(epochs):
        running_l_adv = 0
        running_loss = 0
        for data in dataloader:
            x,y = data
            x.requires_grad = True
            output = model(x)
            x.retain_grad()
            loss = nn.CrossEntropyLoss()

            model.zero_grad()
            l = loss(output,y)
            l.backward(retain_graph = True)
            jacobian = x.grad
            x.requires_grad = False
            print(np.shape(jacobian))
            generated = apt(jacobian)

            #CALCOLO LOSS ADVERSARIAL SFRUTTANDO DEFINIZIONE DI BCE
            real = torch.ones_like(disc(x))
            fake = torch.zeros_like(disc(jacobian))
            bce = nn.BCELoss()
            
            lreal = bce(real, disc(x))
            lfake = bce(fake, disc(generated))
            l_adv = lreal + lfake

            running_l_adv += l_adv.item()

            #CALCOLO LOSS DI TRAINING, NOMI LOSS ASSEGNATI SECONDO I PARAMETRI DEL PAPER

            l_theta = l + u*l_adv   #theta parametro di model 
                                    #psi parametro di apt, da aggiornare minimizzando l_adv
            l_phi = -1*l_adv        #phi parametro di disc, da aggiornare massimizzando l_adv
            

            #Aggiorno parametri
            
            opt_model.zero_grad()
            l_theta.backward(retain_graph= True)
            opt_model.step()
             
            opt_apt.zero_grad()
            l_adv.backward(retain_graph= True)
            opt_apt.step()

            opt_disc.zero_grad()
            l_phi.backward()
            opt_disc.step()
            
            
        print(epoch, running_loss/len(dataloader),running_l_adv/len(dataloader))
    torch.save(model.state_dict(), './JARN_PARAMS.pth')


In [ ]:
epoche= 5
training_Jarn(trainloader, epoche, model, apt,  disc, a= 0.0001, b=0.0001,c = 0.0001, u =1 )

/home/marco/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([256, 3, 32, 32])


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [3, 3, 1, 1]] is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

----
# Exercise 3.3: Experiment with *targeted* adversarial attacks
Implement the targeted Fast Gradient Sign Method to generate adversarial samples that *imitate* samples from a specific class. Evaluate your adversarial samples qualitatively and quantitatively.